In [10]:
!pip install gradio
!pip install transformers
!pip install pronouncing

In [11]:

# Import Required Libraries
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import pronouncing
import random


In [12]:

# Load Pre-Trained GPT Model and Tokenizer
model_name = "gpt2"  # Replace with a fine-tuned model if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [13]:

# Rhyme Scheme Functions
def generate_rhyme_scheme(lines, scheme="monorhyme"):
    rhyme_map = {}
    rhyme_pairs = []

    if scheme == "monorhyme":
        rhyme_word = random.choice(lines[0].split())
        rhymes = pronouncing.rhymes(rhyme_word)
        rhyme_pairs = [rhyme_word if i == 0 else random.choice(rhymes) for i in range(len(lines))]
    elif scheme == "enclosed rhyme":
        rhyme_word1 = random.choice(lines[0].split())
        rhyme_word2 = random.choice(lines[1].split())
        rhymes1 = pronouncing.rhymes(rhyme_word1)
        rhymes2 = pronouncing.rhymes(rhyme_word2)
        rhyme_pairs = [
            rhyme_word1 if i % 4 == 0 else (random.choice(rhymes1) if i % 4 == 3 else rhyme_word2)
            for i in range(len(lines))
        ]

    for i, line in enumerate(lines):
        if i < len(rhyme_pairs):
            lines[i] = line + f" ({rhyme_pairs[i]})"

    return "\n".join(lines)


In [14]:
def filter_repetitions(lines):
    filtered_lines = []
    for line in lines:
        if not filtered_lines or line != filtered_lines[-1]:
            filtered_lines.append(line)
    return filtered_lines


In [15]:
def generate_lyrics(song_title, genre, style, language, rhyme_scheme):
    try:
        # Structured prompt
        prompt = (
            f"Write a {genre.lower()} song titled '{song_title}' in a {style.lower()} style. "
            f"The lyrics should be in {language.lower()}, inspirational, and poetic. "
            "Include a verse and a chorus. The verse should have 4 lines and the chorus should have 4 lines as well. Here's the format:\n\n"
            "Verse 1:\n"
            "Line 1\n"
            "Line 2\n"
            "Line 3\n"
            "Line 4\n\n"
            "Chorus:\n"
            "Line 1\n"
            "Line 2\n"
            "Line 3\n"
            "Line 4\n"
        )

        print(f"Prompt: {prompt}")

        # Tokenize the prompt
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids
        print(f"Input IDs: {input_ids}")

        # Generate the output
        output = model.generate(input_ids, max_length=200, temperature=0.8, top_k=30, top_p=0.85)
        print(f"Model Output: {output}")

        # Check if the output tensor is empty
        if output.numel() == 0:
            return "The model did not generate any output. Please try again with a different prompt."

        # Decode the output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()
        print(f"Raw Generated Text: {generated_text}")

        # Split into lines and filter repetitions
        lines = generated_text.split("\n")
        if not lines:
            return "The generated text was empty. Please try again with a different prompt."
        filtered_lines = filter_repetitions(lines)

        # Apply rhyme scheme if specified
        if rhyme_scheme:
            lyrics = generate_rhyme_scheme(filtered_lines, rhyme_scheme)
        else:
            lyrics = "\n".join(filtered_lines)

        return lyrics
    except Exception as e:
        print(f"Error: {str(e)}")
        return f"An error occurred: {str(e)}"


In [16]:

# Example Rhyme Scheme Output
def rhyme_examples():
    return "\n".join([
        "Monorhyme (AAAA):",
        "Walking through the rain (gain)",
        "Feeling all the pain (gain)",
        "Life's an endless chain (gain)",
        "Breaking every chain (gain)\n",
        "Enclosed Rhyme (ABBA):",
        "The sky is blue (true)",
        "The stars align (design)",
        "A love that's divine (design)",
        "And feelings so true (true)"
    ])


In [17]:

# Define the Gradio Interface
with gr.Blocks() as lyrics_generator:
    gr.Markdown("## 🎵 GPT-Powered Lyrics Generator 🎵")

    with gr.Row():
        song_title = gr.Textbox(label="Song Title", placeholder="Enter a song title...")
        genre = gr.Dropdown(
            ["Pop", "Rock", "Hip-Hop", "Country", "Jazz", "Electronic"], label="Genre", value="Pop"
        )
        style = gr.Dropdown(
            ["Romantic", "Melancholic", "Party", "Inspirational", "Chill"], label="Style", value="Romantic"
        )
        language = gr.Dropdown(
            ["English", "Spanish", "French", "German"], label="Language", value="English"
        )

    rhyme_scheme = gr.Dropdown(
        ["monorhyme", "enclosed rhyme", "none"],
        label="Rhyme Scheme",
        value="monorhyme",
        info="Monorhyme: AAAA\nEnclosed: ABBA"
    )

    example_output = gr.Text(label="Lyrics Example:")
    output = gr.Textbox(label="Generated Lyrics", lines=10)

    generate_button = gr.Button("Generate Lyrics")
    generate_button.click(
        generate_lyrics,
        inputs=[song_title, genre, style, language, rhyme_scheme],
        outputs=output,
    )

    example_button = gr.Button("View Example of Rhyme Schemes")
    example_button.click(
        rhyme_examples,
        inputs=[],
        outputs=example_output
    )




In [ ]:
lyrics_generator.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ecd88c89a887436d27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `30` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected b

Prompt: Write a pop song titled 'Testing Sounds' in a romantic style. The lyrics should be in english, inspirational, and poetic. Include a verse and a chorus. The verse should have 4 lines and the chorus should have 4 lines as well. Here's the format:

Verse 1:
Line 1
Line 2
Line 3
Line 4

Chorus:
Line 1
Line 2
Line 3
Line 4

Input IDs: tensor([[16594,   257,  1461,  3496, 11946,   705, 44154, 27107,     6,   287,
           257, 14348,  3918,    13,   383, 15844,   815,   307,   287, 46932,
            11, 40840,    11,   290, 38273,    13, 40348,   257, 18527,   290,
           257, 26441,    13,   383, 18527,   815,   423,   604,  3951,   290,
           262, 26441,   815,   423,   604,  3951,   355,   880,    13,  3423,
           338,   262,  5794,    25,   198,   198, 13414,   325,   352,    25,
           198, 13949,   352,   198, 13949,   362,   198, 13949,   513,   198,
         13949,   604,   198,   198,  1925, 15125,    25,   198, 13949,   352,
           198, 13949,   362